# Create New Cycle

This notebook creates a new active cycle for your Risk Modeling workflow. 

1. Archives the current active cycle (if any)
2. Creates a new cycle from the template
3. Registers stages and steps in the database

**Note:** Only one cycle can be active at a time.

## Let's get started with library imports

In [1]:
# Import modules directly
import system.database as db
import system.display as display
import system.cycle_manager as cycle_manager
from system.work_context import WorkContext
from system.step import Step

In [2]:
display.print_subheader("Current Cycles")

cycles_df = cycle_manager.get_cycle_status()
if not cycles_df.empty:
    display.display_dataframe(cycles_df, max_rows=10)
else:
    print("No cycles found")

# Check for active cycle
active_cycle = db.get_active_cycle()
if active_cycle:
    print(f"\nActive cycle exists: {active_cycle['cycle_name']}")
    print("This will be archived when creating a new cycle")
    raise Warning("Active cycle detected. Please review before creating a new cycle.")
else:
    display.print_success("No active cycle - ready to create new cycle")

DatabaseError: Query failed: (psycopg2.errors.UndefinedTable) relation "irp_cycle" does not exist
LINE 9:         FROM irp_cycle
                     ^

[SQL: 
        SELECT 
            id,
            cycle_name,
            status,
            created_ts,
            archived_ts,
            created_by
        FROM irp_cycle
        ORDER BY 
            CASE status 
                WHEN 'active' THEN 1
                WHEN 'archived' THEN 2
                ELSE 3
            END,
            created_ts DESC
    ]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [3]:
# Modify this to your desired cycle name
cycle_name = "Q4_2025_Demo-Analysis"

In [7]:
display.print_subheader("Validating Cycle Name")
print(f"Proposed cycle name: {cycle_name}")

# Validate the name
valid, message = cycle_manager.validate_cycle_name(cycle_name)

if valid:
    display.print_success(message)
else:
    display.print_error("\nPlease modify the cycle_name above and re-run this cell")

Proposed cycle name: Q4_2025_Demo-Analysis


In [ ]:
# Creating the cycle
if valid:
    display.print_header(f"Creating Cycle: {cycle_name}")
    
    # Create the cycle
    success = cycle_manager.create_cycle(cycle_name)
    
    if success:
        display.print_header("CYCLE CREATED SUCCESSFULLY")
        
        # Show the new cycle
        new_cycle = db.get_active_cycle()
        if new_cycle:
            display.display_cycle_summary(new_cycle)
        
        # Show directory structure
        from pathlib import Path
        active_dir = Path(f"home/jovyan/workspace/workflows/Active_{cycle_name}")
        
        if active_dir.exists():
            display.print_subheader("Directory Structure")
            print(f"📁 {active_dir}\n")
            
            for item in sorted(active_dir.iterdir()):
                if item.is_dir():
                    print(f"  📂 {item.name}/")
                    if item.name == 'notebooks':
                        for subitem in sorted(item.iterdir()):
                            if subitem.is_dir():
                                print(f"      📂 {subitem.name}/")
    else:
        display.print_error("Failed to create cycle")
else:
    display.print_warning("Please fix the cycle name and try again")


🔨 Creating cycle: Q4_2025_Demo-Analysis
Created directory: /workspace/workflows/Active_Q4_2025_Demo-Analysis
Registered 1 stages with steps

Cycle 'Q4_2025_Demo-Analysis' created successfully


Property,Value
Status,active
Created,2025-10-08 12:34:09.914136+00:00
Created By,notebook_user


📁 /workspace/workflows/Active_Q4_2025_Demo-Analysis

  📂 files/
  📂 logs/
  📂 notebooks/
      📂 Stage_01_Setup/


In [ ]:
display.print_header("Verification")

# Check database
active_cycle = db.get_active_cycle()
if active_cycle:
    display.print_success(f"Active cycle in database: {active_cycle['cycle_name']}")
else:
    display.print_error("No active cycle in database")

# Check filesystem
from pathlib import Path
workflows_path = Path('home/jovyan/workspace/workflows')
active_dirs = [d for d in workflows_path.iterdir() if d.is_dir() and d.name.startswith('Active_')]

if active_dirs:
    display.print_success(f"Active directory found: {active_dirs[0].name}")
else:
    display.print_error("No active directory found")

# Show registered steps
if active_cycle:
    progress_df = cycle_manager.get_cycle_progress(active_cycle['cycle_name'])
    if not progress_df.empty:
        display.print_subheader(f"The cycle has {len(progress_df)} steps")
        display.display_step_progress(progress_df)
    else:
        display.print_warning("No stages and steps - template may be empty")

---